In [1]:
from keras.layers import Bidirectional, Dense, Concatenate, Embedding, Merge, merge, Input, Lambda, LSTM, RepeatVector, TimeDistributed, Layer, Activation, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.layers.advanced_activations import ELU
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras import objectives, backend as K
from keras.models import Model
from keras.datasets import imdb
from scipy import spatial
import tensorflow as tf
import pandas as pd
import numpy as np
import codecs
import csv
import os
import pytrec_eval
from sklearn.utils import shuffle
from Utils import *
from Models import *
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
%%time
q_train, d_train, y_train, q_test, d_test, y_test, qrel, df, df_test, max_len, nb_words = get_data(sup_train_data="1M_qq_log", test_data="JuneFlower", tokenize= "trigram")

b'Skipping line 19898: expected 6 fields, saw 8\nSkipping line 20620: expected 6 fields, saw 8\nSkipping line 38039: expected 6 fields, saw 8\n'


Training Tokeniser
Average query's length: 23
CPU times: user 3min 7s, sys: 2.01 s, total: 3min 9s
Wall time: 3min 8s


In [5]:
%%time
q_train_one_hot = to_categorical(q_train, num_classes=nb_words)
# d_train_one_hot = to_categorical(d_train, num_classes=nb_words)

CPU times: user 5.2 s, sys: 48.9 s, total: 54.1 s
Wall time: 54.1 s


In [6]:
q_train_one_hot = q_train_one_hot.reshape(q_train.shape[0], max_len, nb_words)
# d_train_one_hot = d_train_one_hot.reshape(d_train.shape[0], max_len, nb_words)

In [4]:
q_train_one_hot = np.load("/work/data/numpy/1M_qq_log_q_train_one_hot.npy")

ValueError: cannot reshape array of size 1583346160 into shape (996854,23,50005)

In [ ]:
# np.save("/work/data/numpy/1M_qq_log_d_train_one_hot", d_train_one_hot)

# Read data and Tokenise texts

In [8]:
# http://alexadam.ca/ml/2017/05/05/keras-vae.html
class EMB_LSTM_VAE():
    def __init__(self, vocab_size=50000, max_length=300, latent_rep_size=50):
        self.encoder = None
        self.decoder = None
        self.sentiment_predictor = None
        self.autoencoder = None
        
        self.max_length = max_length
        self.vocab_size = vocab_size
        self.latent_rep_size = latent_rep_size

        x = Input(shape=(max_length,))
        self.x_embed = Embedding(vocab_size, 100, input_length=max_length, mask_zero=True)(x)

        vae_loss, encoded = self._build_encoder(self.x_embed, latent_rep_size=latent_rep_size, max_length=max_length)
        self.encoder = Model(x, encoded)

        encoded_input = Input(shape=(latent_rep_size,))

        decoded = self._build_decoder(encoded_input, vocab_size, max_length)
        self.decoder = Model(encoded_input, decoded)

        self.model = Model(x, self._build_decoder(encoded, vocab_size, max_length))

        self.model.compile(optimizer='Adam',
                                 loss=vae_loss)
        
    def _build_encoder(self, x, latent_rep_size=200, max_length=300, epsilon_std=0.01):
        h = LSTM(500, name='lstm_1')(x)


        def sampling(args):
            z_mean_, z_log_var_ = args
            batch_size = K.shape(z_mean_)[0]
            epsilon = K.random_normal(shape=(batch_size, latent_rep_size), mean=0., stddev=epsilon_std)
            return z_mean_ + K.exp(z_log_var_ / 2) * epsilon

        z_mean = Dense(latent_rep_size, name='z_mean', activation='linear')(h)
        z_log_var = Dense(latent_rep_size, name='z_log_var', activation='linear')(h)

        def vae_loss(x, x_decoded_mean):
            x = K.flatten(x)
            x_decoded_mean = K.flatten(x_decoded_mean)
            xent_loss = max_length * objectives.binary_crossentropy(x, x_decoded_mean)
            kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
            return xent_loss + kl_loss

        return (vae_loss, Lambda(sampling, output_shape=(latent_rep_size,), name='lambda')([z_mean, z_log_var]))
    
    def _build_decoder(self, encoded, vocab_size, max_length):
        
        repeated_context = RepeatVector(max_length)(encoded)
        h = LSTM(500, return_sequences=True, name='dec_lstm_1')(repeated_context)
        decoded = TimeDistributed(Dense(vocab_size, activation='softmax'), name='decoded_mean')(h)
        
        return decoded
    
# emb_lstm_vae.model.fit(X_train_, X_train_, verbose=2, batch_size=batch_size, epochs=10)

In [9]:
# cosine = CosineSim(50)

NameError: name 'CosineSim' is not defined

In [ ]:
emb_lstm_vae = EMB_LSTM_VAE(nb_words, max_len)  
emb_lstm_vae.model.fit(q_train, q_train_one_hot, verbose=2, batch_size=128, epochs=1)

Epoch 1/1


In [ ]:
reader = pd.read_csv(TRAIN_DATA_FILE, chunksize=batch_size, iterator=True, sep="\t", usecols=[0,1,3], names=['label', 'q', 'd'], header=None , error_bad_lines=False)

def sent_generator(reader, tokeniser, batch_size, max_len, feature_num):
    for df in reader:
        q = pad_sequences(l3wt.texts_to_sequences(df.q.tolist()), maxlen=max_len)

        q_one_hot = to_categorical(q, feature_num)   
        q_one_hot = q_one_hot.reshape(batch_size, max_len, feature_num)
        
        
        yield q, q_one_hot
#         print(q_one_hot.shape, q.shape)
#         break

In [80]:
TRAIN_DATA_FILE = "/data/chzho/deepqts/train_data/unifiedclick/join_oneyearsample_2B_training_all_top10"
batch_size = 128
max_len = 10
feature_num = 50005

reader = pd.read_csv(TRAIN_DATA_FILE, chunksize=batch_size, iterator=True, sep="\t", usecols=[0,1,3], names=['label', 'q', 'd'], header=None , error_bad_lines=False)

def sent_generator(reader, tokeniser, batch_size, max_len, feature_num):
    for df in reader:
        q = pad_sequences(tokeniser.texts_to_sequences(df.q.tolist()), maxlen=max_len)

        q_one_hot = to_categorical(q, feature_num)   
        q_one_hot = q_one_hot.reshape(batch_size, max_len, feature_num)
        
        
        yield q, q_one_hot


emb_lstm_vae = EMB_LSTM_VAE(NUM_WORDS, max_len)  
emb_lstm_vae.model.fit_generator(sent_generator(reader, tokeniser, batch_size, max_len, feature_num), steps_per_epoch=100, epochs=2, verbose=2)

Epoch 1/2
 - 1173s - loss: 0.0018
Epoch 2/2
 - 1155s - loss: 0.0011


In [ ]:
emb_lstm_vae = EMB_LSTM_VAE(NUM_WORDS, max_len)  
for i in range(10):
    emb_lstm_vae.model.fit_generator(sent_generator(reader, tokeniser, batch_size, max_len, feature_num), steps_per_epoch=10000, epochs=1, verbose=2)
    pred = get_cosine_sim(emb_lstm_vae.encoder.predict(q1_qq_test), emb_lstm_vae.encoder.predict(q2_qq_test))
    print(auc(df_qq.label.values, pred))

Epoch 1/1
 - 86390s - loss: 7.4557e-04
0.533634281394
Epoch 1/1


In [84]:
# Query-Query data
q1_qq_test = pad_sequences(l3wt.texts_to_sequences(df_qq.q.tolist()), maxlen=max_len)
q2_qq_test = pad_sequences(l3wt.texts_to_sequences(df_qq.d.tolist()), maxlen=max_len)

In [87]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn import metrics

def get_cosine_sim(x, y):
    tmp = []
    for i,j in zip(x,y):
        tmp.append(cosine_similarity(i.reshape(1, -1),j.reshape(1, -1)))
    return np.array(tmp).flatten()

def auc(y_test, pred):
    fpr, tpr, thresholds = metrics.roc_curve(y_test, pred, pos_label=1)
    return metrics.auc(fpr, tpr)        

0.49560808557


In [89]:
pred = get_cosine_sim(q1_qq_test, q2_qq_test)
print(auc(df_qq.label.values, pred))

0.525072827948


In [66]:
# https://nicgian.github.io/text-generation-vae/
batch_size = 100
max_len = 10
emb_dim = 100
latent_dim = 32
intermediate_dim = 96
epsilon_std = 1.0
num_sampled=500
act = ELU()
NUM_WORDS = feature_num

x = Input(batch_shape=(None, max_len))
x_embed = Embedding(NUM_WORDS, emb_dim, input_length=max_len)(x)
h = Bidirectional(LSTM(intermediate_dim, return_sequences=False, recurrent_dropout=0.2), merge_mode='concat')(x_embed)
h = Dropout(0.2)(h)
h = Dense(intermediate_dim, activation='linear')(h)
h = act(h)
h = Dropout(0.2)(h)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_var = args
#     batch_size = z_mean.shape[0]
#     latent_dim = z_mean.shape[1]
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# we instantiate these layers separately so as to reuse them later
repeated_context = RepeatVector(max_len)
decoder_h = LSTM(intermediate_dim, return_sequences=True, recurrent_dropout=0.2)
decoder_mean = TimeDistributed(Dense(NUM_WORDS, activation='linear'))#softmax is applied in the seq2seqloss by tf
h_decoded = decoder_h(repeated_context(z))
x_decoded_mean = decoder_mean(h_decoded)


# placeholder loss
def zero_loss(y_true, y_pred):
    return K.zeros_like(y_pred)


# Custom VAE loss layer
class CustomVariationalLayer(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)
        self.target_weights = tf.constant(np.ones((batch_size, max_len)), tf.float32)

    def vae_loss(self, x, x_decoded_mean):
        #xent_loss = K.sum(metrics.categorical_crossentropy(x, x_decoded_mean), axis=-1)
        labels = tf.cast(x, tf.int32)
        xent_loss = K.sum(tf.contrib.seq2seq.sequence_loss(x_decoded_mean, labels, 
                                                     weights=self.target_weights,
                                                     average_across_timesteps=False,
                                                     average_across_batch=False), axis=-1)
        
        kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        x_decoded_mean = inputs[1]
        print(x.shape, x_decoded_mean.shape)
        loss = self.vae_loss(x, x_decoded_mean)
        self.add_loss(loss, inputs=inputs)
        # we don't use this output, but it has to have the correct shape:
        return K.ones_like(x)

encoder = Model(x, z)

loss_layer = CustomVariationalLayer()([x, x_decoded_mean])
vae = Model(x, [loss_layer])
opt = Adam(lr=0.01) #SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
vae.compile(optimizer='adam', loss=[zero_loss])

(?, 10) (?, 10, 50005)


In [33]:
vae.fit(X_train, X_train, verbose=2, batch_size=batch_size, epochs=10)

Epoch 1/10
 - 354s - loss: 1721.0646
Epoch 2/10
 - 354s - loss: 1531.4780
Epoch 3/10
 - 353s - loss: 1529.2349
Epoch 4/10
 - 349s - loss: 1527.8187
Epoch 5/10
 - 350s - loss: 1526.2863
Epoch 6/10
 - 348s - loss: 1526.0963
Epoch 7/10
 - 348s - loss: 1525.4968
Epoch 8/10
 - 350s - loss: 1525.1179
Epoch 9/10
 - 348s - loss: 1524.6072
Epoch 10/10
 - 350s - loss: 1524.4047


In [44]:
X_train_.shape

(2000, 300)

In [43]:
# train_indices = np.random.choice(np.arange(X_train.shape[0]), 2000, replace=False)
# test_indices = np.random.choice(np.arange(X_test.shape[0]), 1000, replace=False)

# X_train_ = X_train[train_indices]
# y_train_ = y_train[train_indices]

# X_test_ = X_test[test_indices]
# y_test_ = y_test[test_indices]

In [ ]:
temp = np.zeros((X_train.shape[0], MAX_LENGTH, NUM_WORDS))
temp[np.expand_dims(np.arange(X_train.shape[0]), axis=0).reshape(X_train.shape[0], 1), np.repeat(np.array([np.arange(MAX_LENGTH)]), X_train.shape[0], axis=0), X_train] = 1

X_train_one_hot = temp

temp = np.zeros((X_test.shape[0], MAX_LENGTH, NUM_WORDS))
temp[np.expand_dims(np.arange(X_test.shape[0]), axis=0).reshape(X_test.shape[0], 1), np.repeat(np.array([np.arange(MAX_LENGTH)]), X_test.shape[0], axis=0), X_test] = 1

x_test_one_hot = temp

In [ ]:
from keras.utils import to_ca